In [4]:
from PIL import Image
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from tqdm import tqdm
import os, sys
from os import path
from import_augmentation_function import import_fun, aug_fun
import matplotlib.pyplot as plt
import tensorflow as tf 

In [3]:
## IMPORTS ALL IMAGES WITHIN THE FOLDER "imager_dir" INTO TWO ARRAYS, ONE FOR GAUSS POINTS AND ONE FOR IMAGE CROPS ##

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf_pos'
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
aug_data, aug_data_gauss, validation_data, validation_data_gauss = aug_fun(0.1, all_image_array, all_image_array_gauss, img_size)

100%|██████████| 520/520 [00:00<00:00, 1384.68it/s]


In [5]:
#Define GPU device where the code will run on

gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
model_path = base_dir+'Models/'
print('model_path:', model_path)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
model_path: C:\Users\roumba\Documents\Software\deep-eventsModels/


In [6]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], aug_data, aug_data_gauss, batch_size=b) 

Model: ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
42/42 - 17s - loss: 0.7138 - binary_accuracy: 0.5955 - val_loss: 0.6903 - val_binary_accuracy: 0.5006 - 17s/epoch - 394ms/step
Epoch 2/20
42/42 - 6s - loss: 0.6270 - binary_accuracy: 0.5873 - val_loss: 0.6768 - val_binary_accuracy: 0.4476 - 6s/epoch - 147ms/step
Epoch 3/20
42/42 - 6s - loss: 0.4810 - binary_accuracy: 0.6373 - val_loss: 0.6261 - val_binary_accuracy: 0.8894 - 6s/epoch - 146ms/step
Epoch 4/20
42/42 - 6s - loss: -1.0249e-02 - binary_accuracy: 0.7588 - val_loss: 0.7385 - val_binary_accuracy: 0.8876 - 6s/epoch - 146ms/step
Epoch 5/20
42/42 - 6s - loss: -1.0431e+00 - binary_accuracy: 0.8300 - val_loss: 0.7446 - val_binary_accuracy: 0.5843 - 6s/epoch - 146ms/step
Epoch 6/20
42/42 - 6s - loss: -2.1972e+00 - binary_accuracy: 0.8641 - val_loss: 0.6486 - val_binary_accuracy: 0.9530 - 6s/epoch - 147ms/step
Epoch 7/20
42/42 - 6s - loss: -3.0388e+00 - binary_accuracy: 0.8849 - 

ResourceExhaustedError: Graph execution error:

Detected at node 'model_3/concatenate_7/concat' defined at (most recent call last):
    File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_fore

In [ ]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)